In [1]:
import os
import numpy as np
import pandas as pd
from skimage.io import imread
from time import time
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.model_selection import  cross_val_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV, KFold, cross_validate
from sklearn.linear_model import Perceptron
from sklearn.decomposition import PCA
from nilearn import plotting
import nibabel as nib


from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [2]:
## Read data
Working_directory="./" 
df_train = pd.read_csv(Working_directory + 'metaDataTrain.csv') # reading metaDataTrain
df_test = pd.read_csv(Working_directory + 'metaDataTest.csv') # reading metadataTest

#y = df['Malignant'].values # 1 for Melanoma and 0 for healthy
#class_names = ["healthy","melanoma"]

#N,M=X.shape
#print('Number of images: {0}; Number of features per image: {1}'.format(N,M))
#print('Number of healthy nevus: {0}; Number of melanoma: {1}'.format(N-np.sum(y), np.sum(y)))

In [3]:
df_test.columns

Index(['Id', 'Height', 'Weight'], dtype='object')

In [11]:
#On crée un dictionnaire avec toutes nos images train (originales et segmentée)

data_train = {}
for i in range(1,101):
    images = {}
    
    #ED_seg
    img_path = './Train/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ED_seg.nii'
    images["img_ED_seg"] = nib.load(img_path)
    images["data_ED_seg"] = nib.load(img_path).get_fdata()
    
    #ED
    img_path = './Train/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ED.nii'
    images["img_ED"] = nib.load(img_path)
    images["data_ED"] = nib.load(img_path).get_fdata()
    
    #ES_seg
    img_path = './Train/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ES_seg.nii'
    images["img_ES_seg"] = nib.load(img_path)
    images["data_ES_seg"] = nib.load(img_path).get_fdata()
    
    #ES
    img_path = './Train/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ES.nii'
    images["img_ES"] = nib.load(img_path)
    images["data_ES"] = nib.load(img_path).get_fdata()
    
    
    data_train[f'{i:03}'] = images
    
    

In [12]:
#On crée un dictionnaire avec toutes nos images test (originales et segmentée)

data_test = {}
for i in range(101,151):
    images = {}
    
    #ED_seg
    img_path = './Test/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ED_seg.nii'
    images["img_ED_seg"] = nib.load(img_path)
    images["data_ED_seg"] = nib.load(img_path).get_fdata()
    
    #ED
    img_path = './Test/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ED.nii'
    images["img_ED"] = nib.load(img_path)
    images["data_ED"] = nib.load(img_path).get_fdata()
    
    #ES_seg
    img_path = './Test/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ES_seg.nii'
    images["img_ES_seg"] = nib.load(img_path)
    images["data_ES_seg"] = nib.load(img_path).get_fdata()
    
    #ES
    img_path = './Test/' + f'{i:03}' +  '/'+ f'{i:03}' +'_ES.nii'
    images["img_ES"] = nib.load(img_path)
    images["data_ES"] = nib.load(img_path).get_fdata()
    
    
    data_test[f'{i:03}'] = images

In [16]:
pixel03 = []
pixel1 = []
pixel2 = []
for images in data_train.values():
    #pixel distribution for ED
    data_ED_seg = images.get("data_ED_seg")
    background_left_ventricule_ED = np.sum(np.where(data_ED_seg==0, 1, 0)) + np.sum(np.where(data_ED_seg==3, 1, 0))
    Right_ventricle_cavity_ED = np.sum(np.where(data_ED_seg==1, 1, 0))
    Myocardium_ED = np.sum(np.where(data_ED_seg==2, 1, 0))
    
    #pixel distribution for ES
    data_ES_seg = images.get("data_ES_seg")
    background_left_ventricule_ES = np.sum(np.where(data_ES_seg==0, 1, 0)) + np.sum(np.where(data_ES_seg==3, 1, 0)) 
    Right_ventricle_cavity_ES = np.sum(np.where(data_ES_seg==1, 1, 0))
    Myocardium_ES = np.sum(np.where(data_ES_seg==2, 1, 0))
    
    #Difference between both distribution
    pixel03.append(np.abs(background_left_ventricule_ED - background_left_ventricule_ES))
    pixel1.append(np.abs(Right_ventricle_cavity_ED - Right_ventricle_cavity_ES))
    pixel2.append(np.abs(Myocardium_ED - Myocardium_ES))      

df_train = df_train.assign(background_left_ventricule = pixel03, Right_ventricle_cavity = pixel1, Myocardium = pixel2)

In [18]:
pixel03 = []
pixel1 = []
pixel2 = []
for images in data_test.values():
    #pixel distribution for ED
    data_ED_seg = images.get("data_ED_seg")
    background_left_ventricule_ED = np.sum(np.where(data_ED_seg==0, 1, 0)) + np.sum(np.where(data_ED_seg==3, 1, 0))
    Right_ventricle_cavity_ED = np.sum(np.where(data_ED_seg==1, 1, 0))
    Myocardium_ED = np.sum(np.where(data_ED_seg==2, 1, 0))
    
    #pixel distribution for ES
    data_ES_seg = images.get("data_ES_seg")
    background_left_ventricule_ES = np.sum(np.where(data_ES_seg==0, 1, 0)) + np.sum(np.where(data_ES_seg==3, 1, 0)) 
    Right_ventricle_cavity_ES = np.sum(np.where(data_ES_seg==1, 1, 0))
    Myocardium_ES = np.sum(np.where(data_ES_seg==2, 1, 0))
    
    #Difference between both distribution
    pixel03.append(np.abs(background_left_ventricule_ED - background_left_ventricule_ES))
    pixel1.append(np.abs(Right_ventricle_cavity_ED - Right_ventricle_cavity_ES))
    pixel2.append(np.abs(Myocardium_ED - Myocardium_ES))      

df_test = df_test.assign(background_left_ventricule = pixel03, Right_ventricle_cavity = pixel1, Myocardium = pixel2)
print(df_train)

     Id  Category  Height  Weight  background_left_ventricule  \
0     1         2   184.0    95.0                        2022   
1     2         2   160.0    70.0                        1791   
2     3         2   165.0    77.0                         349   
3     4         2   159.0    46.0                         814   
4     5         2   165.0    77.0                        3289   
..  ...       ...     ...     ...                         ...   
95   96         4   186.0    76.0                        4826   
96   97         4   187.0    82.0                        2855   
97   98         4   167.0    50.0                         345   
98   99         4   180.0    80.0                        4019   
99  100         4   165.0    63.0                        3444   

    Right_ventricle_cavity  Myocardium  
0                     3284        1262  
1                     3510        1719  
2                      727         378  
3                     1162         348  
4             

In [19]:
X_train = df_train.loc[:,'Height':'Myocardium'].values # Features
X_test = df_test.loc[:,'Height':'Myocardium'].values # Features
y_train = df_train['Category']

In [20]:
from sklearn.preprocessing import StandardScaler

In [21]:
StdScaler = StandardScaler().fit(X_train)
X_train = StdScaler.transform(X_train)
X_test = StdScaler.transform(X_test)

In [22]:
Tree = DecisionTreeClassifier(random_state=0)
Tree.fit(X_train,y_train)

# Score in the training set
print('Score in the training set is {0}'.format(Tree.score(X_train,y_train)) )
y_pred = Tree.predict(X_test)

Score in the training set is 1.0


In [23]:
sampleSubmission = pd.read_csv(Working_directory + 'SampleSubmission.csv')

In [24]:
sampleSubmission

,Id,Category
0,101,3
1,102,4
2,103,3
3,104,1
4,105,2
5,106,2
6,107,2
7,108,3
8,109,2
9,110,1


In [25]:
dictionnary = {'Id' : np.arange(101,151,1), 'Category' : y_pred}

In [26]:
SampleSubmission = pd.DataFrame(dictionnary)
SampleSubmission.to_csv('SampleSubmission.csv', index=False)